In [26]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

In [27]:
# Hodgkin-Huxley model
# https://en.wikipedia.org/wiki/Hodgkin%E2%80%93Huxley_model
class HHModel:

    class Gate:
        alpha, beta, state = 0, 0, 0

        def update(self, deltaTms):
            alphaState = self.alpha * (1-self.state)
            betaState = self.beta * self.state
            self.state += deltaTms * (alphaState - betaState)

        def setInfiniteState(self):
            self.state = self.alpha / (self.alpha + self.beta)

    ENa, EK, EKleak = 115, -12, 10.6
    ENa, EK, EKleak = 200, -12, 10.6 # Higher ENa
    ENa, EK, EKleak = 115, -24, 10.6 # Lower Ek
    ENa, EK, EKleak = 115, -12, 20.6 # Higher EKleak
    ENa, EK, EKleak = 115, -12, 10.6
    gNa, gK, gKleak = 120, 36, 0.3
    gNa, gK, gKleak = 240, 36, 0.3 # Higher gNa
    gNa, gK, gKleak = 120, 72, 0.3 # Higher gK
    gNa, gK, gKleak = 120, 36, 0.6 # Higher gK
    
    m, n, h = Gate(), Gate(), Gate()
    Cm = 1

    def __init__(self, startingVoltage=0):
        self.Vm = startingVoltage
        self.UpdateGateTimeConstants(startingVoltage)
        self.m.setInfiniteState()
        self.n.setInfiniteState()
        self.h.setInfiniteState()
        self.INa = 0
        self.IK  = 0
        self.IKleak = 0
        self.Isum = 0

    def UpdateGateTimeConstants(self, Vm):
        self.n.alpha = .01 * ((10-Vm) / (np.exp((10-Vm)/10)-1))
        self.n.beta = .125*np.exp(-Vm/80)
        self.m.alpha = .1*((25-Vm) / (np.exp((25-Vm)/10)-1))
        self.m.beta = 4*np.exp(-Vm/18)
        self.h.alpha = .07*np.exp(-Vm/20)
        self.h.beta = 1/(np.exp((30-Vm)/10)+1)

    def UpdateCellVoltage(self, stimulusCurrent, deltaTms):
        self.INa = np.power(self.m.state, 3) * self.gNa * \
                   self.h.state*(self.Vm-self.ENa)
        self.IK = np.power(self.n.state, 4) * self.gK * (self.Vm-self.EK)
        self.IKleak = self.gKleak * (self.Vm-self.EKleak)
        self.Isum = stimulusCurrent - self.INa - self.IK - self.IKleak
        self.Vm += deltaTms * self.Isum / self.Cm

    def UpdateGateStates(self, deltaTms):
        self.n.update(deltaTms)
        self.m.update(deltaTms)
        self.h.update(deltaTms)

    def Iterate(self, stimulusCurrent=0, deltaTms=0.05):
        self.UpdateGateTimeConstants(self.Vm)
        self.UpdateCellVoltage(stimulusCurrent, deltaTms)
        self.UpdateGateStates(deltaTms)

In [28]:
hh = HHModel()
pointCount = 5000
Vm = np.empty(pointCount)
n = np.empty(pointCount)
m = np.empty(pointCount)
h = np.empty(pointCount)
INa = np.empty(pointCount)
IK = np.empty(pointCount)
IKleak = np.empty(pointCount)
Isum = np.empty(pointCount)
times = np.arange(pointCount) * 0.05
stim = np.zeros(pointCount)
stim[2000:3000] = 10

In [29]:
for i in range(len(times)):
    hh.Iterate(stimulusCurrent=stim[i], deltaTms=0.05)
    Vm[i] = hh.Vm
    n[i]  = hh.n.state
    m[i]  = hh.m.state
    h[i]  = hh.h.state
    INa[i] = hh.INa
    IK[i] = hh.IK
    IKleak[i] = hh.IKleak
    Isum[i] = hh.Isum

In [30]:
import os

def create_directory(base_path, new_folder):
    """Ensure the base directory exists and create a new subdirectory for the plots."""
    try:
        os.makedirs(os.path.join(base_path, new_folder), exist_ok=True)
    except OSError as error:
        print(f"Error creating directory {new_folder} in {base_path}: {error}")
        return None
    return os.path.join(base_path, new_folder)

def next_folder_number(base_path):
    """Determine the next folder number by checking existing folders."""
    if not os.path.exists(base_path):
        return 1
    existing_folders = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
    if not existing_folders:
        return 1
    max_number = max([int(folder) for folder in existing_folders if folder.isdigit()])
    return max_number + 1

def save_plot(figure, save_path, file_name):
    """Save the plot to the specified location."""
    figure.savefig(os.path.join(save_path, file_name))
    plt.close(figure)  # Close the figure to free memory

def plot_hodgkin_huxley_neuron_model(times, Vm, stim):
    fig, ax = plt.subplots(figsize=(10,5))
    ax.plot(times, Vm - 70, linewidth=2, label='Vm')
    ax.plot(times, stim - 70, label = 'Stimuli (Scaled)', linewidth=2, color='sandybrown')
    ax.set_ylabel("Membrane Potential (mV)", fontsize=15)
    ax.set_xlabel('Time (msec)', fontsize=15)
    ax.set_xlim([90,160])
    ax.set_title("Hodgkin-Huxley Neuron Model", fontsize=15)
    ax.legend(loc=1)
    return fig

def plot_gating_variables(times, m, h, n):
    fig, ax = plt.subplots(figsize=(10,5))
    ax.plot(times, m, label='m (Na)', linewidth=2)
    ax.plot(times, h, label='h (Na)', linewidth=2)
    ax.plot(times, n, label='n (K)', linewidth=2)
    ax.set_ylabel("Gate state", fontsize=15)
    ax.set_xlabel('Time (msec)', fontsize=15)
    ax.set_xlim([90,160])
    ax.set_title("Hodgkin-Huxley Spiking Neuron Model: Gatings", fontsize=15)
    ax.legend(loc=1)
    return fig

def plot_ion_currents(times, INa, IK, IKleak, Isum):
    fig, ax = plt.subplots(figsize=(10,5))
    ax.plot(times, INa, label='INa', linewidth=2)
    ax.plot(times, IK, label='IK', linewidth=2)
    ax.plot(times, IKleak, label='Ileak', linewidth=2)
    ax.plot(times, Isum, label='Isum', linewidth=2)
    ax.set_ylabel("Current (uA)", fontsize=15)
    ax.set_xlabel('Time (msec)', fontsize=15)
    ax.set_xlim([90,160])
    ax.set_title("Hodgkin-Huxley Spiking Neuron Model: Ion Currents", fontsize=15)
    ax.legend(loc=1)
    return fig

base_path = 'HHplots'
folder_number = next_folder_number(base_path)
subfolder_path = create_directory(base_path, str(folder_number))

if subfolder_path:
    # Generate the Hodgkin-Huxley Neuron Model plot
    fig1 = plot_hodgkin_huxley_neuron_model(times, Vm, stim)
    save_plot(fig1, subfolder_path, 'neuron_model.png')
    
    # Generate the Gating Variables plot
    fig2 = plot_gating_variables(times, m, h, n)
    save_plot(fig2, subfolder_path, 'gating_variables.png')

    # Generate the Ion Currents plot
    fig3 = plot_ion_currents(times, INa, IK, IKleak, Isum)
    save_plot(fig3, subfolder_path, 'ion_currents.png')